**Задание 1**
Импортируйте библиотеки pandas и numpy.
Загрузите "Boston House Prices dataset" из встроенных наборов данных библиотеки sklearn. Создайте датафреймы X и y из этих данных.

In [20]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston

dataset = load_boston()
X = pd.DataFrame(dataset["data"],columns = dataset["feature_names"])
y = pd.DataFrame(dataset["target"])

Разбейте эти датафреймы на тренировочные (X_train, y_train) и тестовые (X_test, y_test) с помощью функции train_test_split так, чтобы размер тестовой выборки составлял 30% от всех данных, при этом аргумент random_state должен быть равен 42.

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

Создайте модель линейной регрессии под названием lr с помощью класса LinearRegression из модуля sklearn.linear_model.
Обучите модель на тренировочных данных (используйте все признаки) и сделайте предсказание на тестовых.
Вычислите R2 полученных предказаний с помощью r2_score из модуля sklearn.metrics.

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

lr = LinearRegression(normalize = True)
lr.fit(X_train, y_train)
y_test_pred = lr.predict(X_test)
R2_lr = r2_score(y_test, y_test_pred)
R2_lr

0.7109203586326285

**Задание 2**
Создайте модель под названием model с помощью RandomForestRegressor из модуля sklearn.ensemble.
Сделайте агрумент n_estimators равным 1000, max_depth должен быть равен 12 и random_state сделайте равным 42.

Обучите модель на тренировочных данных аналогично тому, как вы обучали модель LinearRegression, но при этом в метод fit вместо датафрейма y_train поставьте y_train.values[:, 0], чтобы получить из датафрейма одномерный массив Numpy, так как для класса 
RandomForestRegressor в данном методе для аргумента y предпочтительно применение массивов вместо датафрейма.

Сделайте предсказание на тестовых данных и посчитайте R2. Сравните с результатом из предыдущего задания.
Напишите в комментариях к коду, какая модель в данном случае работает лучше.

In [40]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 1000,
                              max_depth = 12,
                              random_state = 42)
model.fit(X_train, y_train.values[:, 0])
y_test_pred = model.predict(X_test)
R2_rf = r2_score(y_test, y_test_pred)
print("Linear: {:.1f}%\nRandomForest: {:.1f}%".format(R2_lr * 100, R2_rf * 100))
# По оценке среднеквадтратичной ошибки, модель случайного леса работает ощутимо лучше линейной

Linear: 71.1%
RandomForest: 87.6%


**Задание 3**
Вызовите документацию для класса RandomForestRegressor, найдите информацию об атрибуте feature_importances_.
С помощью этого атрибута найдите сумму всех показателей важности, установите, какие два признака показывают наибольшую важность.

In [77]:
from collections import OrderedDict

rf_model_importances = OrderedDict(zip(X_train.columns, model.feature_importances_))
sorted(rf_model_importances.items(), 
       key = lambda item:item[1], 
       reverse = True)
# наиболее существенными признаками выглядят LSTAT и RM. Судя по описанию признаков, это показатель наличия образования / статуса 
# проживающих и среднее количество комнат в доме. Вполне ожидаемо, что данные признаки имеют большое влияние на стоимость жилья.

5       5.21
116    12.04
45     10.21
16      6.58
468    18.13
360     7.79
3       2.94
405    22.98
185    13.15
60     13.15
110    13.00
321     6.87
265    10.45
29     11.98
262     5.91
478    18.03
26     14.81
7      19.15
492    13.35
108    12.27
37      8.77
157     4.59
472    14.36
118    15.37
114    10.45
175     5.33
192     2.87
272     7.73
144    29.29
373    34.77
       ...  
276     6.05
443    18.85
191     4.69
385    30.81
293     8.58
413    20.08
343     7.18
257     5.12
308     4.54
149    21.45
130    12.60
151    13.28
359    12.67
99      6.19
372     8.88
87      8.44
458    16.23
330     9.09
214    29.55
466    17.15
121    14.27
505     7.88
20     21.02
188     4.56
71      9.88
106    18.66
270    13.00
348     5.99
435    23.27
102    10.63
Name: LSTAT, Length: 354, dtype: float64

**Задание 4**
В этом задании мы будем работать с датасетом, с которым мы уже знакомы по домашнему заданию по библиотеке Matplotlib, это датасет Credit Card Fraud Detection.Для этого датасета мы будем решать задачу классификации - будем определять,какие из транзакциции по кредитной карте являются мошенническими.Данный датасет сильно несбалансирован (так как случаи мошенничества относительно редки),так что применение метрики accuracy не принесет пользы и не поможет выбрать лучшую модель.Мы будем вычислять AUC, то есть площадь под кривой ROC.

Импортируйте из соответствующих модулей RandomForestClassifier, GridSearchCV и train_test_split.
Загрузите датасет creditcard.csv и создайте датафрейм df.
С помощью метода value_counts с аргументом normalize=True убедитесь в том, что выборка несбалансирована.Используя метод info, проверьте, все ли столбцы содержат числовые данные и нет ли в них пропусков.Примените следующую настройку, чтобы можно было просматривать все столбцы датафрейма:
pd.options.display.max_columns = 100.
Просмотрите первые 10 строк датафрейма df.
Создайте датафрейм X из датафрейма df, исключив столбец Class.
Создайте объект Series под названием y из столбца Class.

Разбейте X и y на тренировочный и тестовый наборы данных при помощи функции train_test_split, используя аргументы: test_size=0.3, random_state=100, stratify=y.
У вас должны получиться объекты X_train, X_test, y_train и y_test.
Просмотрите информацию о их форме.
Для поиска по сетке параметров задайте такие параметры:
parameters = [{'n_estimators': [10, 15],
'max_features': np.arange(3, 5),
'max_depth': np.arange(4, 7)}]
Создайте модель GridSearchCV со следующими аргументами:
estimator=RandomForestClassifier(random_state=100),
param_grid=parameters,
scoring='roc_auc',
cv=3.
Обучите модель на тренировочном наборе данных (может занять несколько минут).
Просмотрите параметры лучшей модели с помощью атрибута best_params_.
Предскажите вероятности классов с помощью полученнной модели и метода predict_proba.
Из полученного результата (массив Numpy) выберите столбец с индексом 1 (вероятность класса 1) и запишите в массив y_pred_proba. Из модуля sklearn.metrics импортируйте метрику roc_auc_score.
Вычислите AUC на тестовых данных и сравните с результатом,полученным на тренировочных данных, используя в качестве аргументов массивы y_test и y_pred_proba.


In [86]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

pd.options.display.max_columns = 100

df = pd.DataFrame(pd.read_csv("../../Input/creditcard.csv")) # on a github verssion there is no dataset

In [108]:
df.head(10)
df.columns
X = df[[i for i in df.columns if i != "Class"]]
y = pd.Series(df["Class"])

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.3, 
                                                    random_state = 100, 
                                                    stratify = y)
parameters = [{'n_estimators': [10, 15],
                'max_features': np.arange(3, 5),
                'max_depth': np.arange(4, 7)
               }]
Grid_model = GridSearchCV(estimator = RandomForestClassifier(random_state = 100),
                         param_grid = parameters,
                         scoring = 'roc_auc',
                         cv = 3)
Grid_model.fit(X_train, y_train)
Grid_model.best_params_

In [113]:
from sklearn.metrics import roc_auc_score

y_pred_proba = Grid_model.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred_proba)

0.9462664156037157